In [1]:
import torch
import numpy as np


In [2]:
criterion = torch.nn.CrossEntropyLoss()

In [3]:
criterion(torch.tensor([[0.0, 0.0, 1.0, 0.0]]), torch.tensor([2]))

tensor(0.7437)

In [4]:
y = torch.tensor([[1,2,3],[4,5,6]])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [5]:
A = torch.diag_embed(y)

In [6]:
p = torch.unsqueeze(y, 2)

In [7]:
p

tensor([[[1],
         [2],
         [3]],

        [[4],
         [5],
         [6]]])

In [8]:
torch.matmul(p, torch.transpose(p, 1, 2))

tensor([[[ 1,  2,  3],
         [ 2,  4,  6],
         [ 3,  6,  9]],

        [[16, 20, 24],
         [20, 25, 30],
         [24, 30, 36]]])

In [9]:
torch.sqrt(torch.tensor(A, dtype=torch.float))

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([[[1.0000, 0.0000, 0.0000],
         [0.0000, 1.4142, 0.0000],
         [0.0000, 0.0000, 1.7321]],

        [[2.0000, 0.0000, 0.0000],
         [0.0000, 2.2361, 0.0000],
         [0.0000, 0.0000, 2.4495]]])

In [10]:
x = torch.tensor(A, dtype=torch.float)

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [11]:
x = x.cuda()

In [12]:
e, U = torch.linalg.eig(x)

In [17]:
torch.mean(e[:, -1])

tensor(4.5000+0.j, device='cuda:0')

In [22]:
U

tensor([[[1.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 1.+0.j]],

        [[1.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 1.+0.j]]], device='cuda:0')

In [19]:
U.size()

torch.Size([2, 3, 3])

In [18]:
torch.mean(U, dim=-1)

tensor([[0.3333+0.j, 0.3333+0.j, 0.3333+0.j],
        [0.3333+0.j, 0.3333+0.j, 0.3333+0.j]], device='cuda:0')

In [25]:
U.flatten(start_dim=1).size()

torch.Size([2, 9])

In [27]:
torch.stack([U, U], dim=-1).size()

torch.Size([2, 3, 3, 2])

In [2]:
np.random.seed(1)
a = np.random.random(size=10)
a = a / np.sum(a)

In [3]:
np.random.seed(1)
b = np.random.random(size=(20,10))

In [6]:

y_pred = torch.tensor(a).unsqueeze(0)
W = torch.tensor(b).unsqueeze(0)*5

In [12]:
scale=1.0
EPS = 1e-12

In [13]:
A = torch.diag_embed(y_pred)  # (B, cl, cl)
p = torch.unsqueeze(y_pred, 2) # (B, cl, 1)
A = A - torch.matmul(p, torch.transpose(p, 1, 2)) # A-pp^T
A = A + torch.eye(int(A.size()[1]), int(A.size()[2])).unsqueeze(0) * EPS
e, U = torch.linalg.eig(A)
e = torch.clamp(torch.real(e), min=1e-20, max=np.inf)
Sigma = torch.diag_embed(torch.sqrt(e))
L = torch.matmul(torch.real(U), Sigma)
B = torch.matmul(W, L)  # (B, K, C) x (B, C, C) --> (B, K, C)
B_T = torch.transpose(B, 1, 2) # (B, C, K)
C = torch.matmul(B_T, B) # (B, C, C)
C = C + torch.eye(int(C.size()[1]), int(C.size()[2])).unsqueeze(0) * EPS
H_e, _ = torch.linalg.eig(C)
avg_beta = torch.mean(torch.real(H_e[:, -1])) # Eigenvalues. Shape is [..., N]. Sorted in non-decreasing order.
print(avg_beta * scale)

AttributeError: module 'torch.linalg' has no attribute 'eig'

In [54]:
avg_beta

tensor(6.8540e-10, dtype=torch.float64)

In [59]:
e

tensor([[2.0780e-01, 1.5660e-01, 1.2940e-01, 1.1581e-01, 1.0042e-01, 6.5493e-02,
         4.9813e-02, 3.1578e-02, 9.9999e-13, 4.0389e-05]], dtype=torch.float64)

In [58]:
A.size()

torch.Size([1, 10, 10])